In [1]:
import os
from dotenv import load_dotenv
from langchain_groq import ChatGroq
load_dotenv()

c:\Users\akhil\Documents\Agentic_AI_Langchain\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


True

In [2]:
from langchain_chroma import Chroma
from langchain_huggingface import HuggingFaceEmbeddings
groq_api_key = os.getenv("GROQ_API_KEY")

In [3]:
os.environ["HF_TOKEN"] = os.getenv("HF_API_KEY")

In [4]:
llm = ChatGroq(groq_api_key=groq_api_key, model="meta-llama/llama-4-maverick-17b-128e-instruct")

In [5]:
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

In [7]:
from langchain_core.documents import Document
documents = [
    Document(page_content="LangChain is a framework for developing applications powered by language models.", metadata={"source": "langchain_docs"}),
    Document(page_content="Chroma is an open-source embedding database.", metadata={"source": "chroma_docs"}),
    Document(page_content="HuggingFace provides a wide variety of pre-trained models for NLP tasks.", metadata={"source": "huggingface_docs"})
]

In [8]:
vectorstore = Chroma.from_documents(documents,embeddings)

In [12]:
from typing import List
from langchain_core.documents import Document
from langchain_core.runnables import RunnableLambda

retriever = RunnableLambda(vectorstore.similarity_search_with_score).bind(k=1)
retriever.batch(["Chroma","NLP"])

[[(Document(id='83540bfc-4486-4d9d-adf4-a783e8b35baf', metadata={'source': 'chroma_docs'}, page_content='Chroma is an open-source embedding database.'),
   0.9354726672172546)],
 [(Document(id='3535582b-d275-49fc-a80f-891d069bc4bc', metadata={'source': 'huggingface_docs'}, page_content='HuggingFace provides a wide variety of pre-trained models for NLP tasks.'),
   0.961976945400238)]]

In [13]:
retriever1 = vectorstore.as_retriever(
    search_type = "similarity",
    search_kwargs = {"k": 1}
)
retriever1.batch(["Chroma","Language models"])

[[Document(id='83540bfc-4486-4d9d-adf4-a783e8b35baf', metadata={'source': 'chroma_docs'}, page_content='Chroma is an open-source embedding database.')],
 [Document(id='98d8b655-3db3-43ae-baed-63519ac4dfc0', metadata={'source': 'langchain_docs'}, page_content='LangChain is a framework for developing applications powered by language models.')]]

In [15]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough

message = """
Answer this question using the provided context only.
{question}

Context:
{context}
"""
prompt = ChatPromptTemplate.from_messages([("human", message)])

rag_chain = {"context":retriever1,"question":RunnablePassthrough()}| prompt | llm

rag_chain.invoke("What is Chroma?")

AIMessage(content='Chroma is an open-source embedding database.', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 10, 'prompt_tokens': 76, 'total_tokens': 86, 'completion_time': 0.016339967, 'prompt_time': 0.001355543, 'queue_time': 0.051113837, 'total_time': 0.01769551}, 'model_name': 'meta-llama/llama-4-maverick-17b-128e-instruct', 'system_fingerprint': 'fp_d2c1f7e199', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None, 'model_provider': 'groq'}, id='lc_run--bf828133-5d71-453a-9585-6653e6e7a954-0', usage_metadata={'input_tokens': 76, 'output_tokens': 10, 'total_tokens': 86})